# Ollama: Custom Rest client

In [1]:
%%capture output
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

BASE_PATH         = '../..'
API_PATH          = f'{BASE_PATH}/chat-bot-api'
LIB_PATH          = f'{BASE_PATH}/lib'

sys.path.append(LIB_PATH)
sys.path.append(API_PATH)

In [36]:
from rest.ollama import OllamaApiClient
from recommender.chatbot.movie import MovieRecommendationsOutputParser, MovieRecommenderChatBotFactory

## Setup

In [3]:
user_profile = """
User profile:
- Name: Adrian
- Genre: Male
- Age: 32
- Nationality: Argentina
- Studies: Engineering
- Work: Software Engineer
- Movie preferences:
  - Released from 1970
  - Genres: thiller, suspense, science fiction, love, comedy
"""

candidates = """
Candidate movies (with rating):
- Iron Man 3: 3.3
- Star Trek Into Darkness: 3.5
- Man of Steel: 3.1
- Despicable Me 2: 3.4
- Wolverine, The: 2.8
- Kick-Ass 2: 3.0
- Thor: The Dark World: 3.1
- The Lego Movie: 3.6
- Captain America: The Winter Soldier: 3.6
- The Amazing Spider-Man 2: 2.9
- X-Men: Days of Future Past: 3.6
- Mission: Impossible - Rogue Nation: 3.4
- Guardians of the Galaxy: 3.7
- Crippled Avengers: 2.5
- Penguins of Madagascar: 3.4
- Star Wars: Episode V - The Empire Strikes Back: 4.2
- Star Wars: Episode VII - The Force Awakens: 3.2
- Avengers: Age of Ultron: 3.3
- Ant-Man: 3.4
- Fantastic Four: 1.8
- Guardians of the Galaxy 2: 3.4
- Captain America: Civil War: 3.3
- Doctor Strange: 3.5
- X-Men: Apocalypse: 2.9
- Marvel One-Shot: Item 47: 3.0
- Furious 7: 3.1
- Ghost in the Shell: Stand Alone Complex - The Laughing Man: 4.0
- Absolutely Anything: 3.3
- Next Avengers: Heroes of Tomorrow: 5.0
- Iron Man & Captain America: Heroes United: 3.0
- Batman v Superman: Dawn of Justice: 2.6
- Star Trek: The Motion Picture: 3.1
- Iron Man & Hulk: Heroes United: 3.0
- Wonder Woman: 2.0
- Captain Fantastic: 3.8
- Now You See Me 2: 3.5
- LEGO DC Comics Super Heroes: Batman: Be-Leaguered: 2.0
- While You Were Fighting: A Thor Mockumentary: 3.8
- Marvel One-Shot: All Hail the King: 3.0
- Marvel One-Shot: The Consultant: 2.3
- Rogue One: A Star Wars Story: 3.5
- Superheroes: A Never-Ending Battle: 4.0
- xXx: Return of Xander Cage: 3.4
- The Lego Batman Movie: 3.9
- Marvel One-Shot: Agent Carter: 4.0
- Marvel One-Shot: A Funny Thing Happened on the Way to Thor's Hammer: 4.0
- Logan: 3.8
- Kong: Skull Island: 3.1
- Despicable Me 3: 2.7
- Paragraph 78: Film One: 3.0
- X-Files: Fight the Future, The: 3.4
- Avengers, The: 1.8
- Comic Book Confidential: 4.0
- Captain America: 2.0
- Austin Powers: The Spy Who Shagged Me: 2.8
- Mystery Men: 2.9
- Fantastic Four: 2.5
- Sky High: 2.9
- Starman: 3.7
- X-Men: 3.5
- Supergirl: 2.2
- Demolition Man: 3.3
- X-Men: The Last Stand: 3.0
- Punisher, The: 2.4
- Superman Returns: 2.7
- Blankman: 2.0
- Spider-Man 3: 2.7
- Fantastic Four: Rise of the Silver Surfer: 2.2
- Spider-Man: 3.4
- Men in Black II: 2.7
- Superhero Movie: 2.8
- Iron Man: 3.8
- Incredible Hulk, The: 3.1
- Hancock: 3.2
- Watchmen: 3.7
- Cowboy Bebop: The Movie: 3.9
- X2: X-Men United: 3.6
- Punisher: War Zone: 2.8
- Comic Book Villains: 1.0
- Mystery Science Theater 3000: The Movie: 4.1
- X-Men Origins: Wolverine: 3.0
- Star Trek: 3.8
- Fanboys: 2.8
- Avatar: 3.7
- Phantom, The: 3.1
- Iron Man 2: 3.4
- Despicable Me: 3.6
- Green Hornet, The: 2.7
- Thor: 3.1
- Spider-Man 2: 3.5
- X-Men: First Class: 3.5
- Captain America: The First Avenger: 3.2
- Incredibles, The: 3.8
- Avengers, The: 3.6
- Superheroes: 4.0
- Mission: Impossible - Ghost Protocol: 3.6
- Comic-Con Episode IV: A Fan's Hope: 2.0
- Men in Black III: 3.2
- Amazing Spider-Man, The: 3.1
- Rise of the Guardians: 3.7
"""

In [41]:
OllamaApiClient().models()


[
    'gemma:7b',
    'gemma-7b:latest',
    'llama2:7b-chat',
    'llama2-7b-chat:latest',
    'llama3:instruct',
    'llama3-8b-instruct:latest',
    'llama3-rec:latest',
    'mistral:instruct',
    'mistral:latest',
    'mistral-instruct:latest',
    'mixtral:latest'
]

In [57]:
%%time

PROMPT = """
{user_profile}

{user_history}

{candidates}

Recommend {limit} movies from candidate movies list, based on user profile and seen movies.
Return only one list of recommendations with next format:

Number. Title (Release year): Synopsis.
"""

charbot_client = MovieRecommenderChatBotFactory.create(
    prompt = PROMPT,
    model  = 'llama3-rec:latest'
)

result = charbot_client(
    user_profile = user_profile, 
    candidates   = candidates,
    limit        = 10
)

result.metadata['recommendations']

CPU times: user 14.3 ms, sys: 3.63 ms, total: 17.9 ms
Wall time: 4.72 s



[
    {
        'number': 1,
        'title': 'Star wars: episode v - the empire strikes back',
        'description': 'A classic space opera film that explores the complexities of friendship and loyalty as luke skywalker trains with yoda to confront darth vader.',
        'release': '1980'
    },
    {
        'number': 2,
        'title': 'Cowboy bebop: the movie',
        'description': 'An animated sci-fi film set in a futuristic world where bounty hunters navigate the galaxy, exploring themes of identity and purpose.',
        'release': '2001'
    },
    {
        'number': 3,
        'title': 'Watchmen',
        'description': 'A superhero deconstruction film that delves into the complexities of heroism, morality, and the consequences of vigilantism.',
        'release': '2009'
    },
    {
        'number': 4,
        'title': 'Incredibles, the',
        'description': 'An animated superhero film that explores the challenges of balancing family life with superpowers, featuring

In [59]:
dir(result)


[
    '__annotations__',
    '__class__',
    '__delattr__',
    '__dict__',
    '__dir__',
    '__doc__',
    '__eq__',
    '__format__',
    '__ge__',
    '__getattribute__',
    '__getstate__',
    '__gt__',
    '__hash__',
    '__init__',
    '__init_subclass__',
    '__le__',
    '__lt__',
    '__module__',
    '__ne__',
    '__new__',
    '__reduce__',
    '__reduce_ex__',
    '__repr__',
    '__setattr__',
    '__sizeof__',
    '__str__',
    '__subclasshook__',
    '__weakref__',
    'content',
    'metadata'
]